In [1]:
## extract feature from original image and sobel filtered image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy.sparse import dok_matrix,coo_matrix,csr_matrix,vstack,save_npz

import pandas as pd
import sys
from pathlib2 import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import pickle
from joblib import dump, load
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [2]:
def cal_dice(tp,fp,fn):
    return (2*tp)/(2*tp+fp+fn)

def kedney_or_tumor(x):
    y = x.copy()
    y[y==2] = 1
    y[y==3] = 0
    return y

def tumor_only(y):
    x = y.copy()
    x[x==1] = 0
    x[x==3] = 0
    return x



In [3]:
data_train = np.load('data_train.npy')
data_train.shape

(876480949, 6)

In [4]:
np.unique(data_train[:,-1])

array([-1.,  0.,  1.,  2.,  3.], dtype=float32)

In [4]:
(data_train[:,-1]==-1).sum()

575

In [4]:
# index_0 = np.arange(0,len(data_train),25)
index_0 = data_train[:,-1]==0
index_1 = data_train[:,-1]==1
index_2 = data_train[:,-1]==2
index_3 = data_train[:,-1]==3

In [5]:
data_0 = data_train[index_0,:]
data_1 = data_train[index_1,:]
data_2 = data_train[index_2,:]
data_3 = data_train[index_3,:]

In [ ]:
print(data_0.shape)
data_0 = np.unique(data_0,axis = 0)
print(data_0.shape)

In [6]:
data_3.shape

(967, 6)

In [7]:
forest = RandomForestClassifier(n_estimators=0,
                                bootstrap = True,
#                                 max_samples = 0.25
#                                 min_samples_split = 100,
#                                 min_samples_leaf = 50,
                                max_depth=30, 
                                random_state=456,
                                n_jobs=8,
                                max_features=2,
                                criterion='entropy',
                                verbose=2,
#                                 ccp_alpha=0.01,
                                warm_start=True)


In [8]:
slice_num = 25
for i in range(slice_num):
    print(i)
    data_new_slice = np.concatenate((data_0[np.arange(i,len(data_0),slice_num)],
                               data_1[np.arange(i,len(data_1),slice_num)],
                               data_2[np.arange(i,len(data_2),slice_num)],
                               data_3[np.arange(i,len(data_3),slice_num)]),0)

    fea_datasets = data_new_slice[:,:5]
    target_list = data_new_slice[:,5]
    
    forest.n_estimators = forest.n_estimators + 8
    forest.fit(fea_datasets, target_list)
    
    

(35059217, 6)
(24335453, 6)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 8
building tree 2 of 8
building tree 3 of 8
building tree 4 of 8
building tree 5 of 8
building tree 6 of 8
building tree 7 of 8
building tree 8 of 8


[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:  1.0min remaining:  1.7min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.1min finished


(35059217, 6)
(24334862, 6)


...

[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   58.2s remaining:  1.6min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.0min finished


(35059213, 6)
(24334873, 6)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 8
building tree 2 of 8
building tree 3 of 8
building tree 4 of 8
building tree 5 of 8
building tree 6 of 8
building tree 7 of 8building tree 8 of 8



[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   58.4s remaining:  1.6min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.1min finished


In [9]:
dump(forest, 'RF_warmv1.joblib')

['RF_warmv1.joblib']

In [ ]:
# target_list = target_list[~np.isnan(fea_datasets).any(axis=1)]


# fea_datasets = fea_datasets[~np.isnan(fea_datasets).any(axis=1), :]

In [ ]:
y_true_temp = data_train[:,5]
# ypred = bst.predict(fea_datasets)
pred = forest.predict(data_train[:,:5])
# pred = np.argmax(ypred, axis=1)
accuracy = accuracy_score(y_true_temp, pred)

cm1 = confusion_matrix(kedney_or_tumor(y_true_temp), kedney_or_tumor(pred))
cm2 = confusion_matrix(tumor_only(y_true_temp), tumor_only(pred))
if cm1.shape[0]>1:
    dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
if cm2.shape[0]>1:
    dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])
print(accuracy,dice1,dice2)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


In [3]:
forest = load ('RF_warmv1.joblib')

In [4]:

data_val = np.load('data_val.npy')
fea_val = data_val[:,:5]
y_true = data_val[:,5]
y_true[y_true==-1]=0
id_val = data_val[:,6]

result_all = np.empty([63,3])
for i in range(63):
    fea_val_temp = fea_val[id_val==i,:]
    y_true_temp = y_true[id_val==i]
#     ypred = bst.predict(fea_val_temp)
    pred = forest.predict(fea_val_temp)
#     pred = np.argmax(ypred, axis=1)
    accuracy = accuracy_score(y_true_temp, pred)

    cm1 = confusion_matrix(kedney_or_tumor(y_true_temp), kedney_or_tumor(pred))
    cm2 = confusion_matrix(tumor_only(y_true_temp), tumor_only(pred))
    if cm1.shape[0]>1:
        dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
    if cm2.shape[0]>1:
        dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])
    print(i,accuracy,dice1,dice2)
    result_all[i,:] = [accuracy,dice1,dice2]

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   14.2s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:   19.3s finished


0 0.9940830791498687 0.4344044375882102 0.0


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   15.6s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:   20.8s finished


1 0.9921263972340639 0.38403573190113444 0.0


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   13.8s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:   18.7s finished


...

60 0.9951587244325573 0.489774251229325 0.0


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    6.3s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    8.4s finished


61 0.9690224303387674 0.0 0.0


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   17.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:   22.9s finished


62 0.9926146825816207 0.012534818941504178 0.0


In [5]:
result_all.mean(axis=0)

array([9.88604839e-01, 2.89942083e-01, 4.94448757e-06])

In [6]:
np.save('result_warm_v1',result_all)

In [6]:
bst = LGBMClassifier(
    boosting_type='rf',
    max_depth = 30,
    num_leaves = 59,
    n_estimators = 200,
    objective = 'multiclass',
    random_state = 456,
    n_jobs = 8,
    bagging_freq = 8,
    bagging_fraction = 1/25,
    verbose = 2,
#     subsample_freq=0
    
)

In [7]:
data_new = np.concatenate((data_0,
                               data_1,
                               data_2,
                               data_3),0)
# train_data = lgb.Dataset(data_new[:,:5], label=data_new[:,5])

# train_data.save_binary('train.bin')

In [ ]:
bst.fit(data_new[:,:5], data_new[:,5])

[LightGBM] [Warning] bagging_fraction is set=0.04, subsample=1.0 will be ignored. Current value: bagging_fraction=0.04
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.04, subsample=1.0 will be ignored. Current value: bagging_fraction=0.04
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.056681
[LightGBM] [Debug] init for col-wise cost 0.000232 seconds, init for row-wise cost 25.339658 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 8.672695 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 87

In [44]:
train_data = lgb.Dataset(fea_datasets, label=target_list)

train_data.save_binary('train.bin')

[LightGBM] [Info] Saving data to binary file train.bin


In [45]:
train_data = lgb.Dataset('train.bin')

In [54]:
param = {
#         'num_leaves': 31, 
        'max_depth':30,
    'random_state':456,
        'n_estimators':8
#          'min_data_in_leaf':
         'objective': 'multiclass',
#          'num_class':4,
         'boosting_type':'rf',
         'tree_learner ':'data',
         'num_threads':8,
         'device_type':'cpu',
#          'device_type':'gpu',
#         'force_row_wise':True,
#         'force_col_wise':True,
        'bagging_fraction':0.75,
        'verbose': 5,
        'bagging_freq':10}
param['metric'] = 'multi_error'

In [55]:
num_round = 100
bst = lgb.train(param, train_data, num_round)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000402
[LightGBM] [Debug] init for col-wise cost 0.000026 seconds, init for row-wise cost 0.309516 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.250618 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 33252429, number of used features: 5
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score -0.327498
[LightGBM] [Info] Start training from score -1.507155
[LightGBM] [Info] Start training from score -2.852376
[LightGBM] [Info] Start training from score -10.445440
[LightGBM] [Debug] Re-bagging, using 24939836 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree wi

In [ ]:
bst.save_model('model.txt')

In [56]:
y_true_temp = target_list
ypred = bst.predict(fea_datasets)
# pred = forest.predict(fea_datasets)
pred = np.argmax(ypred, axis=1)
accuracy = accuracy_score(y_true_temp, pred)

cm1 = confusion_matrix(kedney_or_tumor(y_true_temp), kedney_or_tumor(pred))
cm2 = confusion_matrix(tumor_only(y_true_temp), tumor_only(pred))
if cm1.shape[0]>1:
    dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
if cm2.shape[0]>1:
    dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])
print(accuracy,dice1,dice2)

0.8671987841850591 0.7946169518805983 0.25499198541406


In [57]:

data_val = np.load('data_val.npy')
fea_val = data_val[:,:5]
y_true = data_val[:,5]
y_true[y_true==-1]=0
id_val = data_val[:,6]

result_all = np.empty([63,3])
for i in range(63):
    fea_val_temp = fea_val[id_val==i,:]
    y_true_temp = y_true[id_val==i]
    ypred = bst.predict(fea_val_temp)
#     pred = forest.predict(fea_val_temp)
    pred = np.argmax(ypred, axis=1)
    accuracy = accuracy_score(y_true_temp, pred)

    cm1 = confusion_matrix(kedney_or_tumor(y_true_temp), kedney_or_tumor(pred))
    cm2 = confusion_matrix(tumor_only(y_true_temp), tumor_only(pred))
    if cm1.shape[0]>1:
        dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
    if cm2.shape[0]>1:
        dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])
    print(i,accuracy,dice1,dice2)
    result_all[i,:] = [accuracy,dice1,dice2]

0 0.980937262341868 0.4115341990699253 0.0
1 0.9661223188211204 0.34219762694999695 0.0
2 0.9545979544841408 0.32969170460444275 0.030787943289368656
3 0.9666676731826966 0.33378370485739317 0.0
4 0.9591026599895942 0.14914337785954593 0.0004139072847682119
5 0.9377257221075052 0.13536389879981242 0.0
6 0.9477721819827977 0.2017217290185971 0.0
7 0.9621853150044727 0.3841732895810019 0.01110994928634914
8 0.9564550679630593 0.42907726979273525 0.14166777879294945
9 0.9753137104000462 0.2969467974101894 0.0
10 0.9568033392434988 0.2301704966641957 0.0
11 0.945257048679019 0.22073542262003165 0.006825676815706726
12 0.9589984431272509 0.30074913960584837 0.05104251938171137
13 0.9411085490637062 0.1580529039883948 0.0022640732736441288
14 0.962994022746488 0.06586075393147069 0.0
15 0.9246585062467143 0.26615306467998645 0.0
16 0.9413917060396833 0.2272559924992514 0.0001679590179996081
17 0.9360976473922903 0.20943369704636902 0.00010535187526337969
18 0.9792753489826643 0.4474235363811

In [51]:
result_all.mean(axis=0)

array([0.92044728, 0.20320874, 0.03876046])

In [ ]:
y_true = y_true[~np.isnan(fea_val).any(axis=1)]
id_val = id_val[~np.isnan(fea_val).any(axis=1)]

fea_val = fea_val[~np.isnan(fea_val).any(axis=1), :]

In [ ]:
result_all = np.empty([63,3])
for i in range(63):
    fea_val_temp = fea_val[id_val==i,:]
    y_true_temp = y_true[id_val==i]
#     ypred = bst.predict(fea_val_temp)
    pred = forest.predict(fea_val_temp)
#     pred = np.argmax(ypred, axis=1)
    accuracy = accuracy_score(y_true_temp, pred)

    cm1 = confusion_matrix(kedney_or_tumor(y_true_temp), kedney_or_tumor(pred))
    cm2 = confusion_matrix(tumor_only(y_true_temp), tumor_only(pred))
    if cm1.shape[0]>1:
        dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
    if cm2.shape[0]>1:
        dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])
    print(i,accuracy,dice1,dice2)
    result_all[i,:] = [accuracy,dice1,dice2]

In [ ]:
result_all.mean(axis=0)

In [ ]:
np.save('result_v2',result_all)

In [ ]:
print(accuracy,dice1,dice2)

In [ ]:
index_vali = np.arange(int(np.round(0.1*len(data))))
data_vali = data[index_vali,:]
                       

In [ ]:
# np.unique((data_vali[:,-1]))

In [ ]:
ypred = bst.predict(data_vali[:,:4])

In [ ]:
ypred

In [ ]:
pred = np.argmax(ypred, axis=1)

In [ ]:
np.unique(pred)

In [ ]:
y_true = data_vali[:,-1]

accuracy = accuracy_score(y_true, pred)

cm1 = confusion_matrix(kedney_or_tumor(y_true), kedney_or_tumor(pred))
cm2 = confusion_matrix(tumor_only(y_true), tumor_only(pred))
if cm1.shape[0]>1:
    dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
if cm2.shape[0]>1:
    dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])

In [ ]:
print(accuracy,dice1,dice2)

In [ ]:
print(accuracy,dice1,dice2)

In [ ]:
model = load ('RF_v1.joblib')

In [ ]:
pred = model.predict(data_vali[:,:4])

In [ ]:
accuracy = accuracy_score(y_true, pred)

cm1 = confusion_matrix(kedney_or_tumor(y_true), kedney_or_tumor(pred))
cm2 = confusion_matrix(tumor_only(y_true), tumor_only(pred))
if cm1.shape[0]>1:
    dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
if cm2.shape[0]>1:
    dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])
print(accuracy,dice1,dice2)

In [ ]:
cm1

In [ ]:
dice1

In [ ]:
dice2

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(fea_datasets, target_list, test_size = 0.4, random_state = 0)
Val_num = int(np.round(0.9*len(data)))

x_train = fea_datasets[Val_num:,:]
x_test = fea_datasets[:Val_num,:]
y_train = target_list[Val_num:]
y_test = target_list[:Val_num]

In [ ]:
# forest = RandomForestClassifier(n_estimators=4, random_state=456,n_jobs=4,max_features=2,criterion='entropy',warm_start=True)
forest = RandomForestClassifier(n_estimators=200,max_depth=10, random_state=456,n_jobs=4,max_features=2,criterion='entropy',warm_start=False)

In [ ]:
x_train[np.isnan(x_train)].shape

In [ ]:
y_train = y_train[~np.isnan(x_train).any(axis=1)]
x_train = x_train[~np.isnan(x_train).any(axis=1), :]


In [ ]:
forest = forest.fit(x_train, y_train)
dump(forest, 'RF_v2.joblib')

In [ ]:
2

In [ ]:
3

In [ ]:
data = pd.DataFrame(data)
data.columns = ['original','mean','mean_sobel','square','label']
data

In [ ]:
np.unique(data.label)

In [ ]:
data.loc[data['label']==-1]

In [ ]:
data.loc[data['label']==-1,['label']]=0

In [ ]:
data.loc[2871179,'label']

In [ ]:
Val_num = int(np.round(0.3*len(data)))
Val_num

In [ ]:
X_train = data[['original','mean','mean_sobel','square']][Val_num:]
y_train = data['label'][Val_num:]

In [ ]:
X_train = csr_matrix(X_train)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(fea_datasets, target_list, test_size = 0.2, random_state = 0)

In [ ]:
X_val = data[['original','mean','mean_sobel','square']][:Val_num]
y_val = data['label'][:Val_num]

In [ ]:
# t=0
dice_best = 0
slice = 248*248
for i in [182]:
#     t = t + 1
#     forest.set_params(n_estimators=t)
#     train_slice = X_train[slice*(i):slice*(i+1)]
    train_slice = data_cpu[['original','mean','mean_sobel','square']][Val_num:Val_num+slice*(i+1)]
#     y_slice = y_train[slice*(i):slice*(i+1)]
    y_slice = data_cpu['label'][Val_num:Val_num+slice*(i+1)]
    forest = forest.fit(train_slice, y_slice)
    print(i)
    
    
#     if np.mod(i,100)== 0:
#         print(i)
#         choose = np.random.randint(0, high=63, size=1, dtype='l')[0]
#         val_slice = data_cpu[['original','mean','mean_sobel','square']][choose*128*248*248:(choose+1)*128*248*248]
#         output_val = forest.predict(val_slice)
#         y_true = data_cpu['label'][choose*128*248*248:(choose+1)*128*248*248]
        
#         accuracy = accuracy_score(y_true, output_val)

#         cm1 = confusion_matrix(kedney_or_tumor(y_true), kedney_or_tumor(output_val))
#         cm2 = confusion_matrix(tumor_only(y_true), tumor_only(output_val))
#         if cm1.shape[0]>1:
#             dice1 = cal_dice(cm1[1,1],cm1[0,1],cm1[1,0])
#         if cm2.shape[0]>1:
#             dice2 = cal_dice(cm2[1,1],cm2[0,1],cm2[1,0])
#         if (dice1+dice2)/2 > dice_best:
#             dice_best = (dice1+dice2)/2
#             dump(forest, 'RF_v1.joblib') 
#             print('save model')
#         print(choose)
#         print(accuracy,dice1,dice2)